In [14]:
import sys
sys.path.append("/home/yuahe/projects/OntoAlign-py")
from ontoalign.embeds import PretrainedBert
from ontoalign.onto import Ontology
import torch
from typing import List
import owlready2
import itertools

In [2]:
onto = Ontology("/home/yuahe/projects/OntoAlign-py/largebio_data/ontologies/fma2nci.small.owl")
classes = list(onto.onto.classes())

In [6]:
lexical_properties = ["label", "comment"]
for i in [5, 325, 761  ]:
    entity = classes[i]
    print(entity.label, entity.name) 
    encoded = " <property> ".join([onto.encode_entity_lexicon(entity, lp) for lp in lexical_properties])
    print(encoded)
    print(onto.parse_entity_lexicon(encoded))
    print("--------")

['Body substance', 'Portion of body substance'] Portion_of_body_substance
body substance <sep> portion of body substance <property> 
(['body substance', 'portion of body substance'], 2)
--------
['Epithelial tissue', 'Epithelium', 'Portion of epithelium', 'Textus epithelialis'] Portion_of_epithelium
epithelial tissue <sep> textus epithelialis <sep> portion of epithelium <sep> epithelium <property> 
(['epithelial tissue', 'textus epithelialis', 'portion of epithelium', 'epithelium'], 4)
--------
['Cervical nerve', 'Cervical nerve tree', 'Cervical spinal nerve', 'Nervus cervicalis'] Cervical_nerve
cervical spinal nerve <sep> cervical nerve tree <sep> cervical nerve <sep> nervus cervicalis <property> 
(['cervical spinal nerve', 'cervical nerve tree', 'cervical nerve', 'nervus cervicalis'], 4)
--------


In [7]:
test = onto.load_iri_lexicon_file("/home/yuahe/projects/OntoAlign-py/largebio_data/onto_labels/fma2nci.small.labels.tsv")

In [17]:
bert = PretrainedBert("emilyalsentzer/Bio_ClinicalBERT")

Load the Pretrained BERT model...


In [25]:
entity_embeds = []
i = 0
for batch in onto.iri_lexicon_batch_generator("/home/yuahe/projects/OntoAlign-py/largebio_data/onto_labels/fma2nci.small.labels.tsv", batch_size=250):
    batch_entity_iris = batch["Entity-IRI"]
    batch_lexicon_sents = []
    batch_lexicon_sizes = []
    for lexicon in batch["Entity-Lexicon"]:
        parsed_lexicon_sents, lexicon_size = onto.parse_entity_lexicon(lexicon)
        batch_lexicon_sents.append(parsed_lexicon_sents)
        batch_lexicon_sizes.append(lexicon_size)
    batch_lexicon_sents = list(itertools.chain.from_iterable(batch_lexicon_sents))
    entity_embeds.append(bert.entity_embeds_from_batch_lexicon("batch_sent_embeds_last_2_mean", batch_lexicon_sizes, batch_lexicon_sents))
    i += 1
    if i == 2:
        break
torch.cat(entity_embeds, dim=0).shape

torch.Size([500, 768])